# The BigFlow tutorial part 2

### What you will lean in part 2
* Automatically generating a Airflow DAG from workflow.
* Deploying DAG.

### Your workflow
Lets clean up notebook from part 1.

In [ ]:
import bigflow as bf

In [ ]:
PROJECT_ID = ''  # put you own project ID

In [ ]:
dataset = bf.Dataset(
    project_id=PROJECT_ID,
    dataset_name='transaction_aggregates',
    internal_tables=['user_transaction_metrics'],
    external_tables={
        'transactions': '{}.transactions.transactions'.format(PROJECT_ID)
    })

In [ ]:
create_user_transaction_metrics_table = dataset.create_table('''
CREATE TABLE IF NOT EXISTS user_transaction_metrics (
    user_id STRING,
    metric_name STRING,
    metric_value STRING,
    
    partition_timestamp TIMESTAMP)
PARTITION BY DATE(partition_timestamp)
''')

In [ ]:
write_truncate_user_transaction_value = dataset.write_truncate('user_transaction_metrics', 
'''
SELECT 
    TIMESTAMP('{dt}') as partition_timestamp,
    user_id,
    CAST(sum(CAST(transaction_value as FLOAT64)) as string) as metric_value,
    'USER_TRANSACTION_VALUE' as metric_name
FROM `{transactions}`
WHERE DATE(partition_timestamp) = '{dt}'
GROUP BY user_id
''')

In [ ]:
write_append_user_transaction_count = dataset.write_append('user_transaction_metrics', 
'''
SELECT 
    TIMESTAMP('{dt}') as partition_timestamp,
    user_id,
    CAST(count(*) as string) as metric_value,
    'USER_TRANSACTION_COUNT' as metric_name
FROM `{transactions}`
WHERE DATE(partition_timestamp) = '{dt}'
GROUP BY user_id
''')

In [ ]:
workflow = bf.Workflow(definition=[
    create_user_transaction_metrics_table.to_job(),
    write_truncate_user_transaction_value.to_job(),
    write_append_user_transaction_count.to_job()])

### Deployment
Now, go to `deploy.ipynb` and run it. Then, go back here.

### What just happened
You generated a ready to deploy DAG. `workflow.zip` is the result. All you need to do is to drag-and-drop `workflow.zip` to DAGs folder (you can find the link to the DAGs folder inside the details of your Composer instance). Generated zip is a standard Airflow DAG (https://airflow.apache.org/concepts.html?highlight=zip#packaged-dags).

### More features
If you want to learn more, go to part 3.